# 根据学生的日常行为，学习并预测该学生的成绩等级‘a’,'b','c'类
 现在的代码预测分数是很低，甚至是负数

In [2]:
import numpy as np
# import matplotlib as mpl
# import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [3]:
path = './yktcj.csv' # 一卡通成绩
data = pd.read_csv(path,sep = ',')
data.head(10)

# del data[0]

,Unnamed: 0,XH,午餐平均消费,早餐次数,洗澡花销,洗澡次数,成绩等级
0,0,201400001,9.150545,29,12.39,3,15.0
1,1,201400003,10.498831,137,307.44,109,15.0
2,2,201400004,9.234351,5,44.45,14,15.0
3,3,201400005,9.738125,18,56.42,20,10.0
4,4,201400007,12.804906,101,103.39,31,20.0
5,5,201400008,13.117500,2,6.44,2,10.0
6,6,201400009,6.760000,38,194.32,38,10.0
7,7,201400045,8.540625,2,178.71,34,20.0
8,8,201400046,11.330233,46,130.27,43,10.0
9,9,201400047,11.862500,6,161.42,45,10.0


In [4]:
data['成绩等级'].value_counts()
pd.cut(data['成绩等级'],[8,20,30,45],labels=['c','b','a'],retbins =True)

(0        c
 1        c
 2        c
 3        c
 4        c
 5        c
 6        c
 7        c
 8        c
 9        c
 10       c
 11       c
 12       c
 13       c
 14       c
 15       c
 16       c
 17       c
 18       c
 19       c
 20       c
 21       c
 22       c
 23       c
 24       c
 25       c
 26       c
 27       c
 28       c
 29       c
         ..
 10583    c
 10584    c
 10585    c
 10586    c
 10587    c
 10588    c
 10589    c
 10590    b
 10591    b
 10592    c
 10593    c
 10594    c
 10595    c
 10596    c
 10597    b
 10598    c
 10599    c
 10600    c
 10601    c
 10602    b
 10603    b
 10604    c
 10605    c
 10606    c
 10607    c
 10608    c
 10609    b
 10610    c
 10611    c
 10612    c
 Name: 成绩等级, Length: 10613, dtype: category
 Categories (3, object): [c < b < a], array([ 8, 20, 30, 45]))

In [5]:
# data.drop(columns(['Unnamed：0']),axis = 1)
xt = pd.DataFrame(data[['XH','午餐平均消费','早餐次数','洗澡花销','洗澡次数']])
xt.head(10)

,XH,午餐平均消费,早餐次数,洗澡花销,洗澡次数
0,201400001,9.150545,29,12.39,3
1,201400003,10.498831,137,307.44,109
2,201400004,9.234351,5,44.45,14
3,201400005,9.738125,18,56.42,20
4,201400007,12.804906,101,103.39,31
5,201400008,13.117500,2,6.44,2
6,201400009,6.760000,38,194.32,38
7,201400045,8.540625,2,178.71,34
8,201400046,11.330233,46,130.27,43
9,201400047,11.862500,6,161.42,45


In [11]:
yt = pd.Series(data['成绩等级'])


# # def cjdj():
    
    
yt.head(10)

0    15.0
1    15.0
2    15.0
3    10.0
4    20.0
5    10.0
6    10.0
7    20.0
8    10.0
9    10.0
Name: 成绩等级, dtype: float64

In [13]:
cv_params = {'learning_rate': [0.1,0.12,0.14]}
other_params = {'n_estimators': 37, 'max_depth': 2, 'min_child_weight': 1, 'seed': 0,
                'reg':'linear','num_class':3}
                      # 'multi':'softmax','num_class':3}
# 'colsample_bytree': 0.8,'subsample': 0.8,
model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model,param_grid=cv_params,scoring = 'r2',cv =3,verbose=1,n_jobs = 4)
optimized_GBM.fit(xt,yt)
evaluat_result = optimized_GBM.grid_scores_
print('每轮迭代运行的结果：{0}'.format(evaluat_result))
print('参数的最佳取值{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))


Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


XGBoostError: [17:36:40] src/objective/regression_obj.cu:65: Check failed: preds.Size() == info.labels_.Size() (21225 vs. 7075) : labels are not correctly providedpreds.size=21225, label.size=7075
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x0000000118b3d579 dmlc::LogMessageFatal::~LogMessageFatal() + 57
  [bt] (1) 2   libxgboost.dylib                    0x0000000118bcad81 xgboost::obj::RegLossObj<xgboost::obj::LinearSquareLoss>::GetGradient(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*) + 625
  [bt] (2) 3   libxgboost.dylib                    0x0000000118b397ef xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*) + 1311
  [bt] (3) 4   libxgboost.dylib                    0x0000000118b58314 XGBoosterUpdateOneIter + 180
  [bt] (4) 5   libffi.6.dylib                      0x000000010433e884 ffi_call_unix64 + 76
  [bt] (5) 6   ???                                 0x